## Import Libraries

In [4]:
import pandas as pd
import numpy as np
from pathlib import Path

In [5]:
pd.set_option('display.max_rows', None, 'display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [6]:
!pwd

/home/xavi/Documents/CreditRisk/data/curated


In [9]:
file: Path = Path('/home/xavi/Documents/CreditRisk/data/curated/dataset.parquet')

In [10]:
df: pd.DataFrame = pd.read_parquet(file, engine='pyarrow')

### Depedent variable:

In [12]:
df['loan_status'].value_counts(dropna=False)

loan_status
Current                                                224226
Fully Paid                                             184739
Charged Off                                             42475
Late (31-120 days)                                       6900
In Grace Period                                          3146
Does not meet the credit policy. Status:Fully Paid       1988
Late (16-30 days)                                        1218
Default                                                   832
Does not meet the credit policy. Status:Charged Off       761
Name: count, dtype: int64